## test to generatefeatures from S1, S2, Meteo and DEM data <br>

In [1]:
import openeo
from extentmapping.openeo.processing import generate_master_feature_cube
from extentmapping.config import get_job_options, get_collection_options, get_advanced_options
from extentmapping.utils import laea20km_id_to_extent, reproj_bbox_to_ll, bbox_area
import os
import json

C:\Users\BUCHHORM\Anaconda3\envs\extent\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### connect to the backend

In [2]:
backend = 'terrascope'
#backend = 'development'    # for current tests

# Make connection
if backend == 'terrascope':
    connection = openeo.connect("https://openeo.vito.be").authenticate_oidc()
elif backend == 'development':
    connection = openeo.connect("https://openeo-dev.vito.be").authenticate_oidc()
else:
    print('currently no specific connections to backends like creodias and sentinelhub are setup.')
    print('use standard entry point')
    connection = openeo.connect("https://openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


### init the job, processing and collection option depending on provider and task

In [3]:
job_options = get_job_options(provider=backend, task='feature_generation')
collection_options = get_collection_options(provider=backend)
# reset S1, DEM and Agera5 to only get Sentinel2 data cube
processing_options = get_advanced_options(provider=backend)

In [4]:
# just print for an overview
print(f'job_options: {job_options}')
print(f'collection_options: {collection_options}')
print(f'processing_options: {processing_options}')

job_options: {'driver-memory': '4G', 'driver-memoryOverhead': '4G', 'driver-cores': '2', 'executor-memory': '3G', 'executor-memoryOverhead': '2G', 'executor-cores': '2', 'max-executors': '50', 'soft-errors': 'true'}
collection_options: {'S2_collection': 'SENTINEL2_L2A', 'METEO_collection': 'AGERA5', 'S1_collection': 'SENTINEL1_GRD', 'DEM_collection': 'COPERNICUS_30'}
processing_options: {'provider': 'terrascope', 's1_orbitdirection': 'DESCENDING', 'target_crs': 3035, 'resolution': 10.0, 'time_interpolation': False, 'ts_interval': 'dekad', 'SLC_masking_algo': 'mask_scl_dilation', 'optical_vi_list': ['NDVI', 'AVI', 'CIRE', 'NIRv', 'NDMI', 'NDWI', 'BLFEI', 'MNDWI', 'NDVIMNDWI', 'S2WI', 'S2REP', 'IRECI'], 'radar_vi_list': ['VHVVD', 'VHVVR', 'RVI'], 'S2_scaling': [0, 10000, 0, 1.0], 'S1_db_rescale': True, 'append': True}


### specify the space and time context

In [5]:
# the time context is given by start and end date
start = '2020-01-01'
end = '2021-01-01'   # the end is always exclusive

In [6]:
# the space context is defined as a bounding box dictionary with south,west,north,east and crs
# we take as example the AOI of Daniele's test for habitat mapping
AOI = {'east': 610240, 'south': 5689760, 'west': 600000, 'north': 5700000, 'crs': 'EPSG:32631'}
# print some area info
bbox_area(AOI)

area of AOI in km2: 104.8576


### run over the AOI and process

In [7]:
test_num = 9

# define the output folder root
out_root = os.path.normpath(r'C:\Users\BUCHHORM\OneDrive - VITO\Documents\Project_work\PEOPLE-EA\habitat_mapping\feature_generation')
out_root = os.path.join(out_root, f'test_v{str(test_num)}')
os.makedirs(out_root, exist_ok=True)

In [9]:
print(f'**** processing AOI')
# that is the master feature cube attempt which is currently failing
feature_cube = generate_master_feature_cube(connection,
                                            AOI,
                                            start,
                                            end,
                                            **collection_options,
                                            **processing_options)

# run it
try:
    job = feature_cube.execute_batch(title=f'master feature generation for AOI - test {str(test_num)}',
                                  description='Generation of features via OpenEO',
                                  out_format='GTiff',
                                  job_options=job_options)

    # Get the results and save to output file
    results = job.get_results()

    outputfile = os.path.join(out_root, f'master_features_cube_openeo_AOI_test-v{str(test_num)}.tif')
    results.download_file(outputfile)
    # get metadata and job info
    with open(outputfile.replace('.tif','.json'), "w") as outfile:
        json.dump(results.get_metadata(),outfile)
    with open(outputfile.replace('.tif','-job.json'), "w") as outfile:
        json.dump(job.describe_job(),outfile)

except Exception as e:
    print(e)

**** processing AOI
0:00:00 Job 'j-231115b21c8a461d837c585c95b10dfe': send 'start'
0:00:16 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:00:22 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:00:28 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:00:36 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:00:47 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:01:00 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:01:16 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:01:35 Job 'j-231115b21c8a461d837c585c95b10dfe': queued (progress N/A)
0:01:59 Job 'j-231115b21c8a461d837c585c95b10dfe': running (progress N/A)
0:02:29 Job 'j-231115b21c8a461d837c585c95b10dfe': running (progress N/A)
0:03:07 Job 'j-231115b21c8a461d837c585c95b10dfe': running (progress N/A)
0:03:54 Job 'j-231115b21c8a461d837c585c95b10dfe': running (progress N/A)
0:04:52 Job 'j-231115b21c8a461d837c585c95b10dfe':

In [ ]:
# load data and check the last processed file
import xarray
ts = xarray.load_dataset(outputfile)
ts